In [ ]:
from itertools import combinations
import random

In [ ]:
from itertools import combinations
from collections import defaultdict

def all_possible_worlds_flexible(n, friends, rivals):
    world_to_number = defaultdict(int)
    number_to_world = defaultdict(list)
    all_cards = [i + 1 for i in range(n)]
    counter = 0

    # Initialize data structures to track the mapping of worlds to numbers
    world_to_number = {}
    number_to_world = {}

    # Total number of cards that rivals will hold
    total_rivals_cards = sum(rivals)

    # Generate all distributions of cards to the players in 'friends' and 'rivals'
    def generate_worlds(cards_left, current_distribution, index, total_distribution):
        nonlocal counter
        if index == len(current_distribution):
            # Once all friends are dealt with, deal all remaining cards to rivals as one group
            if len(cards_left) == total_rivals_cards:
                rival_cards = tuple(sorted(cards_left))
                world = tuple(tuple(sorted(x)) for x in current_distribution) + (rival_cards,)
                world_to_number[world] = counter
                number_to_world[counter] = world
                counter += 1
            return

        num_cards = total_distribution[index]
        for chosen_cards in combinations(cards_left, num_cards):
            new_cards_left = list(cards_left)
            for card in chosen_cards:
                new_cards_left.remove(card)
            # Create a new distribution for the next recursive call
            new_distribution = current_distribution[:]
            new_distribution[index] = chosen_cards
            # Recursively process the next player
            generate_worlds(new_cards_left, new_distribution, index + 1, total_distribution)

    # Start the recursive process
    total_players_distribution = friends
    initial_distribution = [()] * len(total_players_distribution)  # Start with empty tuples
    generate_worlds(all_cards, initial_distribution, 0, total_players_distribution)

    return world_to_number, number_to_world, counter

In [ ]:
from collections import defaultdict

def get_suggested_worlds(friends_cards, number_to_world):
    # This dictionary will store the worlds for each friend by index
    friends_worlds = defaultdict(list)

    # Iterate through all the worlds stored in number_to_world
    for number, world in number_to_world.items():
        # Check each friend's current cards against their portion in each world
        for i, friend_cards in enumerate(friends_cards):
            if set(friend_cards) == set(world[i]):  # If the friend's cards in the world match the given cards
                friends_worlds[i].append(number)  # Store the world number for this friend

    return friends_worlds

In [ ]:
from collections import defaultdict

def get_all_possible_worlds_grouped(n, actual_cards, player_number, number_to_world):
    # Create a dictionary to group worlds by the cards of the specified player
    worlds_grouped_by_player = defaultdict(list)

    # Iterate through each world in the provided number_to_world dictionary
    for world_number, world in number_to_world.items():
        player_cards = world[player_number]  # Adjust for zero-based index
        # Convert player's cards to tuple for consistent key usage
        key = tuple(player_cards)

        # Group worlds by the cards of the specified player
        worlds_grouped_by_player[key].append(world_number)

    # Convert grouped worlds from dict to list of lists
    worlds_grouped = list(worlds_grouped_by_player.values())

    return worlds_grouped

In [ ]:
def get_edges(player_worlds_numbered):
    edges = defaultdict(list)
    for group in player_worlds_numbered:
        for i in range (len(group)):
            for k in range (len(group)):
                if i != k:
                    edges[group[i]].append(group[k])
    return edges

In [ ]:
def get_graph_matrix(player_worlds_numbered, world_to_number):
    edges = get_edges(player_worlds_numbered, world_to_number)
    result = []
    for i in range (len(edges)):
        tmp = []
        for k in range (len(edges)):
            if k in edges[i]:
                tmp.append(1)
            else:
                tmp.append(0)
        result.append(tmp)
    return result

In [ ]:
def check_what_to_delete(suggested, friend, to_delete):
  for group in friend:
    for suggested_world in suggested:
      if suggested_world in group:
        for world in group:
          if world != suggested_world and world not in to_delete:
            to_delete.append(world)

In [ ]:
def win(edges, real):
    if real in edges:
        return edges[real] == []

In [ ]:
def delete_world(current, edges, graph):
    for v in edges.keys():
        if current in edges[v]:
            edges[v].remove(current)
    if current in edges.keys():
        del edges[current]
    if graph.has_node(current):
        graph.remove_node(current)

def turn(current, all_edges, all_graph):
    for i in range(len(all_edges)):
        delete_world(current, all_edges[i], all_graph[i])

In [ ]:
def generate_real_world_scenario(n, friends, rivals):
    # Generate a list of all cards
    real_world = [i + 1 for i in range(n)]
    random.shuffle(real_world)

    # Accumulate indices based on the number of cards per friend and rival
    indices = [0]
    for count in friends + rivals:
        indices.append(indices[-1] + count)

    # Split the shuffled cards among friends and rivals
    hands = [tuple(sorted(real_world[indices[i]:indices[i + 1]])) for i in range(len(indices) - 1)]

    # Separate hands into friends and rivals
    friends_hands = hands[:len(friends)]
    rivals_hands = hands[len(friends):]

    # Combine all rival hands into one tuple if needed (depends on how you want to handle rivals)
    if len(rivals_hands) > 1:
        # Flatten the list of tuples for rivals into a single tuple, if they are to be grouped
        combined_rivals = tuple(sorted(sum(rivals_hands, ())))
    else:
        combined_rivals = rivals_hands[0] if rivals_hands else ()

    # Prepare the full distribution as a tuple
    real = tuple(friends_hands) + (combined_rivals,)

    return real, tuple(friends_hands)

In [ ]:
def game(n, friends, rivals):
    world_to_number, number_to_world, total_worlds = all_possible_worlds_flexible(n, friends, rivals)

    real, friends_cards = generate_real_world_scenario(n, friends, rivals)
    real_numbered = world_to_number[real]

    suggested_worlds = get_suggested_worlds(friends_cards, number_to_world)

    worlds_grouped = defaultdict(list)
    all_edges = defaultdict(dict)
    all_graph = defaultdict(nx.Graph)
    for i in range(len(friends) + 1):
        worlds_grouped[i] = get_all_possible_worlds_grouped(n, real, i, number_to_world)
        all_edges[i] = get_edges(worlds_grouped[i])
        all_graph[i] = create_graph_from_dict(all_edges[i])

    all_turns = defaultdict(list)
    for i in range (len(friends)):
        for k in range (len(friends)):
            check_what_to_delete(suggested_worlds[i], worlds_grouped[k], all_turns[i])

    current = 0

    while all(not win(all_edges[i], real_numbered) for i in range(len(friends) + len(rivals))):
        for num in range(len(friends)):
            if current < len(all_turns[i]):
                turn(all_turns[i][current], all_edges, all_graph)
        current += 1

    if any(win(all_edges[i], real_numbered) for i in range(len(friends), len(friends) + len(rivals))):
        return 0
    else:
        return 1

In [ ]:
def generate_m_games(m, n, friends, rivals):
  result = 0

  for i in range(m):
    result += game(n, friends, rivals)

  return result

In [ ]:
print("Input the amount of games: ")
m = int(input())
print("Input the number of cards overall: ")
n = int(input())
print("Input the amount of cards of friends players divided by space: ")
friends = list(map(int, input().split()))
print("Input the amount of cards of rival players divided by space: ")
rivals = list(map(int, input().split()))

if sum(friends) + sum(rivals) != n:
    print("Invalid data")
    exit(404)

generate_m_games(m, n, friends, rivals)